## Text to NoSQL (MongoDB) Queries

- Building Q&A systems of SQL databases requires executing model-generated SQL queries
- Make sure your database have only Read-Only access to prevent any data loss

In [1]:
from dotenv import load_dotenv
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

load_dotenv()

True

In [2]:
from langchain_ollama import ChatOllama 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough 
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.tools import tool


llm = ChatOllama(model='llama3.2:3b', base_url='http://localhost:11434')
llm.invoke('hi')

AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2024-11-01T09:04:10.7783177Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1864811800, 'load_duration': 1683035900, 'prompt_eval_count': 26, 'prompt_eval_duration': 49020000, 'eval_count': 8, 'eval_duration': 130476000}, id='run-e7284580-af63-4022-9fbc-ab1616485511-0', usage_metadata={'input_tokens': 26, 'output_tokens': 8, 'total_tokens': 34})

In [3]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

### Chains

- convert the question into a SQL query;
- execute the query;
- use the result to answer the original question.

In [3]:
prompt = """
You are a MongoDB expert. Given an input question, create a syntactically correct MongoDB query to run using MongoDB's aggregation framework or find() method, as appropriate. Then, look at the results of the query and return the answer to the input question.

Unless the user specifies in the question a specific number of examples to obtain, limit the query to at most 5 results using the limit() function. You can order the results using sort() to return the most relevant or recent data in the database.

Never query for all fields in a document. Only include the fields needed to answer the question in the projection. Use the exact field names as they appear in the collection to avoid errors. Be cautious not to reference fields that do not exist in the collection. If the question involves today's date, use new Date() to represent the current date.

Use the following format:

Question: Question here
MongoDBQuery: MongoDB Query to run
MongoDBResult: Result of the MongoDBQuery
Answer: Final answer here

Only use the following collections:
{collection_info}

Question: {input}
"""

template = ChatPromptTemplate.from_template(prompt)
template


ChatPromptTemplate(input_variables=['collection_info', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['collection_info', 'input'], input_types={}, partial_variables={}, template="\nYou are a MongoDB expert. Given an input question, create a syntactically correct MongoDB query to run using MongoDB's aggregation framework or find() method, as appropriate. Then, look at the results of the query and return the answer to the input question.\n\nUnless the user specifies in the question a specific number of examples to obtain, limit the query to at most 5 results using the limit() function. You can order the results using sort() to return the most relevant or recent data in the database.\n\nNever query for all fields in a document. Only include the fields needed to answer the question in the projection. Use the exact field names as they appear in the collection to avoid errors. Be cautious not to reference fields tha

In [5]:
# llm
question = "How many customer are there"
response = chain.invoke({"question": question})
print(response)

SELECT COUNT(DISTINCT "CustomerId") FROM Customer


### Extract Correct Query from LLM

In [6]:
from scripts.llm import ask_llm
from langchain_core.runnables import chain


In [7]:
@chain
def get_correct_sql_query(input):
    context = input['context']
    question = input['question']
    intruction = """
        Use above context to fetch the correct SQL query for following question
        {}

        Do not enclose query in ```sql and do not write preamble and explanation.
        You MUST return only single SQL query.
    """.format(question)

    response = ask_llm(context=context, question=intruction)

    return response

In [8]:
get_correct_sql_query.invoke({'context':response, 'question':question})

'SELECT COUNT(DISTINCT "CustomerId") FROM Customer'

### Final SQL Query Chain 

In [9]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.runnables import RunnableLambda

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)

chain = (
        {'context': write_query, 'question': RunnablePassthrough()}
        |get_correct_sql_query
        | execute_query
        )

chain.invoke({"question": "How many customers are there"})

'[(59,)]'

### MySQL Query Execution with Agents

- It can answer questions based on the databases' schema as well as on the databases' content (like describing a specific table).
- It can recover from errors by running a generated query, catching the traceback and regenerating it correctly.
- It can query the database as many times as needed to answer the user question.
- It will save tokens by only retrieving the schema from relevant tables.


In [10]:
# %pip install --upgrade --quiet langgraph


In [11]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001B2FF80FA10>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001B2FF80FA10>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001B2FF80FA10>),
 QuerySQLCheckerTool(description='Use this tool to 

In [12]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(content=SQL_PREFIX)

In [18]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, state_modifier=system_message, debug=False)

In [29]:
question = "How many customers are there?"
question = "How many unique customers are there?"

agent_executor.invoke({"messages": [HumanMessage(content=question)]})

{'messages': [HumanMessage(content='How many unique customers are there?', additional_kwargs={}, response_metadata={}, id='23c55d89-e9d8-4857-b45f-2f40eeb34f39'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2024-11-01T08:56:21.9810875Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'sql_db_query_checker', 'arguments': {'query': 'SELECT COUNT(DISTINCT customer_id) FROM customers'}}}, {'function': {'name': 'sql_db_query', 'arguments': {'query': 'SELECT COUNT(DISTINCT customer_id) FROM customers'}}}, {'function': {'name': 'sql_db_query', 'arguments': {'query': 'SELECT COUNT(DISTINCT customer_id) FROM customers'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 1697946000, 'load_duration': 27551200, 'prompt_eval_count': 717, 'prompt_eval_duration': 425081000, 'eval_count': 82, 'eval_duration': 1211688000}, id='run-8c9f37a0-970c-4f95-bc7e-26e7466ca4d9-0', tool_calls=[{'name': 'sql_

In [32]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content=question)]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2024-11-01T08:57:01.4031595Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'sql_db_list_tables', 'arguments': {}}}, {'function': {'name': 'sql_db_query_checker', 'arguments': {'query': 'SELECT COUNT(DISTINCT customer_id) FROM customers'}}}, {'function': {'name': 'sql_db_query', 'arguments': {'query': 'SELECT DISTINCT customer_id FROM customers'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 1530986300, 'load_duration': 20751000, 'prompt_eval_count': 717, 'prompt_eval_duration': 553597000, 'eval_count': 66, 'eval_duration': 953152000}, id='run-540f4135-be41-4db7-8d43-f4dd38e179b9-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': '9f8cd972-2da3-4182-9152-bd8e22702601', 'type': 'tool_call'}, {'name': 'sql_db_query_checker', 'args': {'query': 'SELECT COUNT(DISTINCT customer_id) FROM customers'}, '

## NoSQL Query Generation